In [1]:
import math
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly
from plotly import tools
from plotly.offline import iplot
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

from pandas.core.frame import DataFrame
from matplotlib.font_manager import FontProperties 
import matplotlib

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D

import datetime
import numpy as geek 

Using TensorFlow backend.


In [3]:
n_input = 7

#上週三到本週二
trainDateStart = '2017-01-04'
trainDateEnd = '2019-01-01'

preDateStart = '2019-01-02'
preDateEnd = '2019-09-24'

predictPeakLoad = pd.read_csv('predictPeakLoad_週三到週二_1106_MAPE_2_278(訓練資料2017_2018).csv')

### 載入資料

In [5]:
df = pd.read_csv('Data/elec_merge_20160101_20190930.csv')
#將日期欄位轉為 datetime 格式。
df['日期'] = pd.to_datetime(df['日期'], format='%Y-%m-%d')

#選擇要使用的欄位
selected_features = ['工業用電(百萬度)', 'isHoliday_shift', 'dayOfYear', 'weekUpdate_shift', 'Temp_Taipei_shift', '尖峰負載(MW)']

#補星期幾
#2016年01月01日 -> 星期五 ， 2016年01月02日 -> 星期六
#2019年09月29日 -> 星期日 ， 2019年09月30日 -> 星期一
df['week'] = np.nan
week = [7, 1, 2, 3, 4, 5, 6]
while len(week)<(len(df['week'])-4):
     week = np.concatenate((week, [7, 1, 2, 3, 4, 5, 6]), axis = None)
week = np.concatenate([[5,6], week, [7,1]])

df['week'] = week
df['week_update'] = week

In [ ]:
#2016年
week_column = 'week_update'
df.at[df['日期']=='2016-01-01', week_column]= 7
df.at[df['日期']=='2016-01-30', week_column]= 5

df.at[df['日期']=='2016-02-07', week_column]= 7
df.at[df['日期']=='2016-02-08', week_column]= 7
df.at[df['日期']=='2016-02-09', week_column]= 7
df.at[df['日期']=='2016-02-10', week_column]= 7
df.at[df['日期']=='2016-02-11', week_column]= 7
df.at[df['日期']=='2016-02-12', week_column]= 7

df.at[df['日期']=='2016-02-28', week_column]= 7 #不變
df.at[df['日期']=='2016-02-29', week_column]= 7
df.at[df['日期']=='2016-03-01', week_column]= 1


df.at[df['日期']=='2016-04-04', week_column]= 7
df.at[df['日期']=='2016-04-05', week_column]= 1
df.at[df['日期']=='2016-04-06', week_column]= 1

df.at[df['日期']=='2016-05-01', week_column]= 7 #不變

df.at[df['日期']=='2016-06-04', week_column]= 5

df.at[df['日期']=='2016-06-08', week_column]= 5
df.at[df['日期']=='2016-06-09', week_column]= 7
df.at[df['日期']=='2016-06-10', week_column]= 7

df.at[df['日期']=='2016-09-10', week_column]= 5

df.at[df['日期']=='2016-09-14', week_column]= 5
df.at[df['日期']=='2016-09-15', week_column]= 7
df.at[df['日期']=='2016-09-16', week_column]= 7

df.at[df['日期']=='2016-10-10', week_column]= 6
df.at[df['日期']=='2016-10-11', week_column]= 1

In [ ]:
#2017年
week_column = 'week_update'
df.at[df['日期']=='2017-01-02', week_column]= 7
df.at[df['日期']=='2017-01-03', week_column]= 1

df.at[df['日期']=='2017-01-26', week_column]= 5
df.at[df['日期']=='2017-01-27', week_column]= 7
df.at[df['日期']=='2017-01-28', week_column]= 7
df.at[df['日期']=='2017-01-29', week_column]= 7
df.at[df['日期']=='2017-01-30', week_column]= 7
df.at[df['日期']=='2017-01-31', week_column]= 7
df.at[df['日期']=='2017-02-01', week_column]= 7
df.at[df['日期']=='2017-02-02', week_column]= 1

df.at[df['日期']=='2017-02-18', week_column]= 5

df.at[df['日期']=='2017-02-27', week_column]= 7
df.at[df['日期']=='2017-02-28', week_column]= 6
df.at[df['日期']=='2017-03-01', week_column]= 1

df.at[df['日期']=='2017-04-03', week_column]= 7
df.at[df['日期']=='2017-04-04', week_column]= 6
df.at[df['日期']=='2017-04-05', week_column]= 1

df.at[df['日期']=='2017-05-01', week_column]= 6
df.at[df['日期']=='2017-05-02', week_column]= 1

df.at[df['日期']=='2017-05-29', week_column]= 7
df.at[df['日期']=='2017-05-30', week_column]= 6
df.at[df['日期']=='2017-05-31', week_column]= 1

df.at[df['日期']=='2017-06-03', week_column]= 5
df.at[df['日期']=='2017-09-30', week_column]= 5

df.at[df['日期']=='2017-10-03', week_column]= 5
df.at[df['日期']=='2017-10-04', week_column]= 7
df.at[df['日期']=='2017-10-05', week_column]= 1

df.at[df['日期']=='2017-10-09', week_column]= 7
df.at[df['日期']=='2017-10-10', week_column]= 6
df.at[df['日期']=='2017-10-11', week_column]= 1

In [ ]:
#2018年
week_column = 'week_update'
df.at[df['日期']=='2018-01-01', week_column]= 7
df.at[df['日期']=='2018-01-02', week_column]= 1

df.at[df['日期']=='2018-02-14', week_column]= 5
df.at[df['日期']=='2018-02-15', week_column]= 7
df.at[df['日期']=='2018-02-16', week_column]= 7
df.at[df['日期']=='2018-02-17', week_column]= 7
df.at[df['日期']=='2018-02-18', week_column]= 7
df.at[df['日期']=='2018-02-19', week_column]= 7
df.at[df['日期']=='2018-02-20', week_column]= 7
df.at[df['日期']=='2018-02-21', week_column]= 1


df.at[df['日期']=='2018-02-27', week_column]= 5
df.at[df['日期']=='2018-02-28', week_column]= 7
df.at[df['日期']=='2018-03-01', week_column]= 1

df.at[df['日期']=='2018-03-31', week_column]= 5

df.at[df['日期']=='2018-04-04', week_column]= 6
df.at[df['日期']=='2018-04-05', week_column]= 7
df.at[df['日期']=='2018-04-06', week_column]= 7
df.at[df['日期']=='2018-04-07', week_column]= 7

df.at[df['日期']=='2018-04-30', week_column]= 5
df.at[df['日期']=='2018-05-01', week_column]= 6
df.at[df['日期']=='2018-05-02', week_column]= 1

df.at[df['日期']=='2018-06-18', week_column]= 7
df.at[df['日期']=='2018-06-19', week_column]= 1

df.at[df['日期']=='2018-09-24', week_column]= 7
df.at[df['日期']=='2018-09-25', week_column]= 1

df.at[df['日期']=='2018-10-09', week_column]= 5
df.at[df['日期']=='2018-10-10', week_column]= 6
df.at[df['日期']=='2018-10-11', week_column]= 1

df.at[df['日期']=='2018-12-22', week_column]= 5
df.at[df['日期']=='2018-12-31', week_column]= 7

In [ ]:
#2019年
week_column = 'week_update'
df.at[df['日期']=='2019-01-01', week_column]= 7
df.at[df['日期']=='2019-01-02', week_column]= 1

df.at[df['日期']=='2019-01-19', week_column]= 5

df.at[df['日期']=='2019-02-04', week_column]= 7
df.at[df['日期']=='2019-02-05', week_column]= 7
df.at[df['日期']=='2019-02-06', week_column]= 7
df.at[df['日期']=='2019-02-07', week_column]= 7
df.at[df['日期']=='2019-02-08', week_column]= 7
# df.at[df['日期']=='2019-02-09', week_column]= 7

df.at[df['日期']=='2019-02-23', week_column]= 5

df.at[df['日期']=='2019-02-27', week_column]= 5
df.at[df['日期']=='2019-02-28', week_column]= 7
df.at[df['日期']=='2019-03-01', week_column]= 7
# df.at[df['日期']=='2019-03-02', week_column]= 7

df.at[df['日期']=='2019-04-03', week_column]= 5
df.at[df['日期']=='2019-04-04', week_column]= 7
df.at[df['日期']=='2019-04-05', week_column]= 7
# df.at[df['日期']=='2019-04-06', week_column]= 7

df.at[df['日期']=='2019-04-30', week_column]= 5
df.at[df['日期']=='2019-05-01', week_column]= 6
df.at[df['日期']=='2019-05-02', week_column]= 1

df.at[df['日期']=='2019-06-06', week_column]= 5
df.at[df['日期']=='2019-06-07', week_column]= 7
# df.at[df['日期']=='2019-06-08', week_column]= 7

df.at[df['日期'] == '2019-09-12', week_column] = 5
df.at[df['日期'] == '2019-09-13', week_column] = 7  

### 載入大氣資料

In [ ]:
atmosphericFactor = pd.read_csv('Data/FromCODIS_clean_daily_ByRegion(ElectRatio)_20160101_20191031.csv')

#位移溫度
atmosphericFactor['Temp_Taipei_shift'] = atmosphericFactor['Temp_Taipei'].shift(-7)
atmosphericFactor['Temp_Taichung_shift'] = atmosphericFactor['Temp_Taichung'].shift(-7)
atmosphericFactor['Temp_Kao_shift'] = atmosphericFactor['Temp_Kao'].shift(-7)

atmosphericFactor['Temp_avg_shift'] = atmosphericFactor['Temp_avg'].shift(-7)

#位移雨量
atmosphericFactor['Precp_Taipei_shift'] = atmosphericFactor['Precp_Taipei'].shift(-7)
atmosphericFactor['Precp_Taichung_shift'] = atmosphericFactor['Precp_Taichung'].shift(-7)
atmosphericFactor['Precp_Kao_shift'] = atmosphericFactor['Precp_Kao'].shift(-7)

atmosphericFactor['Precp_ratio_shift'] = (atmosphericFactor['Precp_Taipei']*atmosphericFactor['ElectRatio_north'] + atmosphericFactor['Precp_Taichung']*atmosphericFactor['ElectRatio_central'] + atmosphericFactor['Precp_Kao']*atmosphericFactor['ElectRatio_south']).shift(-7)
atmosphericFactor['Precp_avg_shift'] = (atmosphericFactor['Precp_Taipei_shift'] + atmosphericFactor['Precp_Taichung_shift'] + atmosphericFactor['Precp_Kao_shift'])/3 


#位移氣壓
atmosphericFactor['StnPres_Taipei_shift'] = atmosphericFactor['StnPres_Taipei'].shift(-7)
atmosphericFactor['StnPres_Taichung_shift'] = atmosphericFactor['StnPres_Taichung'].shift(-7)
atmosphericFactor['StnPres_Kao_shift'] = atmosphericFactor['StnPres_Kao'].shift(-7)

#位移濕度
atmosphericFactor['RH_Taipei_shift'] = atmosphericFactor['RH_Taipei'].shift(-7)
atmosphericFactor['RH_Taichung_shift'] = atmosphericFactor['RH_Taichung'].shift(-7)
atmosphericFactor['RH_Kao_shift'] = atmosphericFactor['RH_Kao'].shift(-7)

#位移風速
atmosphericFactor['WS_Taipei_shift'] = atmosphericFactor['WS_Taipei'].shift(-7)
atmosphericFactor['WS_Taichung_shift'] = atmosphericFactor['WS_Taichung'].shift(-7)
atmosphericFactor['WS_Kao_shift'] = atmosphericFactor['WS_Kao'].shift(-7)

atmosphericFactor = atmosphericFactor[(atmosphericFactor['Date']>='2016-01-01') & (atmosphericFactor['Date']<='2019-09-30')]

df = pd.concat([df, atmosphericFactor.iloc[:,1:]], axis=1, sort = False)

In [ ]:
atmosphericFactor[['Date', 
                   'Precp_Taipei', 'ElectRatio_north',
                   'Precp_Taichung', 'ElectRatio_central',
                   'Precp_Kao', 'ElectRatio_south', 
                   'Precp_Taipei_shift', 'Precp_ratio_shift']]

In [ ]:
df['PrecpTaipeiShift_interval'] = np.nan
df['PrecpTaipeiShift_interval'] = df['PrecpTaipeiShift_interval'].fillna(0)

df.loc[df['Precp_Taipei_shift']>=80, 'PrecpTaipeiShift_interval'] = 1
df.loc[df['Precp_Taipei_shift']>=200, 'PrecpTaipeiShift_interval'] = 2
df.loc[df['Precp_Taipei_shift']>=350, 'PrecpTaipeiShift_interval'] = 3
df.loc[df['Precp_Taipei_shift']>=500, 'PrecpTaipeiShift_interval'] = 4

In [ ]:
df[(df['日期']>='2017-01-01') & (df['日期']<='2019-09-30')].corr()

### 處理 ＆ 位移 其他欄位
要預測未來7天的值，故將其他欄位左移7天，並手動補足剩餘欄位的值。

In [ ]:
numberOfDays_2016 = len(pd.date_range(start='2016-01-01', end='2016-12-31'))
numberOfDays_2017 = len(pd.date_range(start='2017-01-01', end='2017-12-31'))
numberOfDays_2018 = len(pd.date_range(start='2018-01-01', end='2018-12-31'))
numberOfDays_2019 = len(pd.date_range(start='2019-01-01', end='2019-09-30'))

numberOfDays = np.concatenate((np.arange(1, numberOfDays_2016+1), np.arange(1, numberOfDays_2017+1), np.arange(1,numberOfDays_2018 +1), np.arange(1,numberOfDays_2019 +1)),axis=0)
df['dayOfYear'] = numberOfDays

In [ ]:
df['isHoliday_shift'] = df['isHoliday'].shift(-7)
df.loc[1362:1365, ("isHoliday_shift")] = 0
df.loc[1366:1367, ("isHoliday_shift")] = 1
df.loc[1368, ("isHoliday_shift")] = 0
# df[-10:]

In [ ]:
df['尖峰負載(MW)_ori'] = df['尖峰負載(MW)']
df['尖峰負載(MW)'] = df['尖峰負載(MW)'].shift(-7)

#往前位移七天
preDateEnd_ = datetime.datetime.strptime(preDateEnd, "%Y-%m-%d")
preDateEnd_7ago = datetime.datetime(preDateEnd_.year, preDateEnd_.month, preDateEnd_.day) + datetime.timedelta(days=-7)
preDateEnd_7ago = preDateEnd_7ago.strftime("%Y-%m-%d")
preDateEnd_7ago

df.at[(df['日期']>=preDateStart) & (df['日期']<=preDateEnd), '尖峰負載(MW)']= geek.nan
df.at[(df['日期']>=preDateStart) & (df['日期']<=preDateEnd_7ago), '尖峰負載(MW)']= (predictPeakLoad['尖峰負載(MW)_predict'][7:].tolist())

for idx, val in enumerate(df['尖峰負載(MW)']):
    if(pd.isnull(val)):
        newValue = (df.loc[idx-7]['尖峰負載(MW)_ori'] + df.loc[idx-14]['尖峰負載(MW)_ori'])/2
        df.loc[idx, '尖峰負載(MW)'] = newValue

In [ ]:
df['weekUpdate_shift'] = df['week_update'].shift(-7)

df.loc[1362, ("weekUpdate_shift")] = 2
df.loc[1363, ("weekUpdate_shift")] = 3
df.loc[1364, ("weekUpdate_shift")] = 4
df.loc[1365, ("weekUpdate_shift")] = 5
df.loc[1366, ("weekUpdate_shift")] = 6
df.loc[1367, ("weekUpdate_shift")] = 7
df.loc[1368, ("weekUpdate_shift")] = 1
# df[-10:]

In [ ]:
df.corr()

### 將資料集切割為 train set 與 test set

In [ ]:
def split_dataset(data, split_num):
    split_num_start = df[df['日期']== trainDateStart].index.item()
    split_num = df[df['日期']== preDateStart].index.item()
    split_num_end = df[df['日期']== preDateEnd].index.item()
    train, test = data[split_num_start:split_num], data[split_num:split_num_end+1]
    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train) / 7))
    test = np.array(np.split(test, len(test) / 7))
    return train, test